In [1]:
import math
import tkinter as tk
from tkinter import ttk, scrolledtext

class HeadcountCalculatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Headcount Calculator")
        self.root.geometry("600x700")

        self.main_frame = ttk.Frame(self.root, padding="10")
        self.main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

        ttk.Label(self.main_frame, text="Headcount Calculator", font=("Helvetica", 16, "bold")).grid(row=0, column=0, columnspan=2, pady=10)

        ttk.Label(self.main_frame, text="Channel Type:").grid(row=1, column=0, sticky=tk.W, pady=5)
        self.channel_var = tk.StringVar()
        self.channel_menu = ttk.Combobox(self.main_frame, textvariable=self.channel_var, values=["voice", "chat", "email"], state="readonly")
        self.channel_menu.grid(row=1, column=1, sticky=(tk.W, tk.E), pady=5)
        self.channel_menu.bind("<<ComboboxSelected>>", self.update_fields)

        self.inputs = {}
        row = 2
        for label, default in [
            ("Number of Interactions per Hour (Volume):", "100"),
            ("Average Handling Time in Seconds (AHT):", "300"),
            ("Shrinkage Percentage (e.g., 30):", "30"),
            ("Time Interval in Seconds (e.g., 3600):", "3600"),
            ("Maximum Occupancy Percentage (e.g., 85):", "85")
        ]:
            ttk.Label(self.main_frame, text=label).grid(row=row, column=0, sticky=tk.W, pady=5)
            self.inputs[label] = ttk.Entry(self.main_frame)
            self.inputs[label].insert(0, default)
            self.inputs[label].grid(row=row, column=1, sticky=(tk.W, tk.E), pady=5)
            row += 1
        self.service_level_label = ttk.Label(self.main_frame, text="Service Level Percentage (e.g., 80):")
        self.service_level_label.grid(row=row, column=0, sticky=tk.W, pady=5)
        self.inputs["Service Level"] = ttk.Entry(self.main_frame)
        self.inputs["Service Level"].insert(0, "80")
        self.inputs["Service Level"].grid(row=row, column=1, sticky=(tk.W, tk.E), pady=5)
        row += 1

        self.asa_label = ttk.Label(self.main_frame, text="Average Speed of Answer in Seconds (ASA):")
        self.asa_label.grid(row=row, column=0, sticky=tk.W, pady=5)
        self.inputs["ASA"] = ttk.Entry(self.main_frame)
        self.inputs["ASA"].insert(0, "20")
        self.inputs["ASA"].grid(row=row, column=1, sticky=(tk.W, tk.E), pady=5)
        row += 1

        self.concurrency_label = ttk.Label(self.main_frame, text="Concurrent Chats per Agent (Concurrency):")
        self.concurrency_label.grid(row=row, column=0, sticky=tk.W, pady=5)
        self.inputs["Concurrency"] = ttk.Entry(self.main_frame)
        self.inputs["Concurrency"].insert(0, "3")
        self.inputs["Concurrency"].grid(row=row, column=1, sticky=(tk.W, tk.E), pady=5)
        row += 1

        self.efficiency_label = ttk.Label(self.main_frame, text="Efficiency Percentage (e.g., 85):")
        self.efficiency_label.grid(row=row, column=0, sticky=tk.W, pady=5)
        self.inputs["Efficiency"] = ttk.Entry(self.main_frame)
        self.inputs["Efficiency"].insert(0, "85")
        self.inputs["Efficiency"].grid(row=row, column=1, sticky=(tk.W, tk.E), pady=5)
        row += 1

        self.calculate_button = ttk.Button(self.main_frame, text="Calculate", command=self.calculate)
        self.calculate_button.grid(row=row, column=0, columnspan=2, pady=10)

        self.result_area = scrolledtext.ScrolledText(self.main_frame, width=50, height=10, wrap=tk.WORD)
        self.result_area.grid(row=row+1, column=0, columnspan=2, pady=10)

        self.update_fields()

    def update_fields(self, event=None):
        channel = self.channel_var.get()
        if channel in ["voice", "chat"]:
            self.service_level_label.grid()
            self.inputs["Service Level"].grid()
            self.asa_label.grid()
            self.inputs["ASA"].grid()
        else:
            self.service_level_label.grid_remove()
            self.inputs["Service Level"].grid_remove()
            self.asa_label.grid_remove()
            self.inputs["ASA"].grid_remove()

        if channel == "chat":
            self.concurrency_label.grid()
            self.inputs["Concurrency"].grid()
        else:
            self.concurrency_label.grid_remove()
            self.inputs["Concurrency"].grid_remove()

        if channel == "email":
            self.efficiency_label.grid()
            self.inputs["Efficiency"].grid()
        else:
            self.efficiency_label.grid_remove()
            self.inputs["Efficiency"].grid_remove()

    def validate_positive_float(self, value, field_name):
        try:
            val = float(value)
            if val <= 0:
                raise ValueError(f"{field_name} must be greater than 0.")
            return val
        except ValueError as e:
            if "must be greater than 0" in str(e):
                raise
            raise ValueError(f"Invalid input for {field_name}. Please enter a valid number.")

    def validate_percentage(self, value, field_name):
        try:
            val = float(value)
            if val < 0 or val > 100:
                raise ValueError(f"{field_name} must be between 0 and 100.")
            return val / 100
        except ValueError as e:
            if "must be between 0 and 100" in str(e):
                raise
            raise ValueError(f"Invalid input for {field_name}. Please enter a valid number.")

    def calculate(self):
        try:
            channel = self.channel_var.get()
            if not channel:
                raise ValueError("Please select a channel type.")

            volume = self.validate_positive_float(self.inputs["Number of Interactions per Hour (Volume):"].get(), "Volume")
            aht = self.validate_positive_float(self.inputs["Average Handling Time in Seconds (AHT):"].get(), "AHT")
            shrinkage = self.validate_percentage(self.inputs["Shrinkage Percentage (e.g., 30):"].get(), "Shrinkage")
            interval = self.validate_positive_float(self.inputs["Time Interval in Seconds (e.g., 3600):"].get(), "Time Interval")
            max_occupancy = self.validate_percentage(self.inputs["Maximum Occupancy Percentage (e.g., 85):"].get(), "Maximum Occupancy")

            if channel in ["voice", "chat"]:
                service_level = self.validate_percentage(self.inputs["Service Level"].get(), "Service Level")
                asa = self.validate_positive_float(self.inputs["ASA"].get(), "ASA")
            else:
                service_level = 0.8
                asa = 20

            if channel == "chat":
                concurrency = self.validate_positive_float(self.inputs["Concurrency"].get(), "Concurrency")
            else:
                concurrency = 1

            if channel == "email":
                efficiency = self.validate_percentage(self.inputs["Efficiency"].get(), "Efficiency")
            else:
                efficiency = 0.85

            hc, occupancy = self.calculate_required_hc(channel, volume, aht, service_level, asa, concurrency, efficiency, shrinkage, interval, max_occupancy)

            self.result_area.delete(1.0, tk.END)
            self.result_area.insert(tk.END, "Calculating Required Headcount with Occupancy Rate:\n")
            self.result_area.insert(tk.END, "-" * 40 + "\n")
            if channel == 'voice':
                self.result_area.insert(tk.END, f"Voice (Erlang C): {hc} agents, Occupancy Rate: {occupancy:.2f}%\n")
            elif channel == 'chat':
                self.result_area.insert(tk.END, f"Chat (Erlang C with Concurrency): {hc} agents, Occupancy Rate: {occupancy:.2f}%\n")
            else:
                self.result_area.insert(tk.END, f"Email (Linear Method): {hc} agents, Occupancy Rate: {occupancy:.2f}%\n")

        except ValueError as e:
            self.result_area.delete(1.0, tk.END)
            self.result_area.insert(tk.END, f"Error: {str(e)}\n")

    def erlang_c(self, traffic_intensity, num_agents):
        if num_agents == 0:
            return 1.0
        
        numerator = (traffic_intensity ** num_agents) / math.factorial(num_agents)
        denominator = sum((traffic_intensity ** k) / math.factorial(k) for k in range(num_agents)) + \
                      (traffic_intensity ** num_agents) / math.factorial(num_agents) * (num_agents / (num_agents - traffic_intensity))
        
        if num_agents <= traffic_intensity:
            return 1.0
        
        return numerator / denominator

    def calculate_occupancy(self, traffic_intensity, num_agents):
        if num_agents == 0:
            return 100.0
        occupancy = (traffic_intensity / num_agents) * 100
        return min(occupancy, 100.0)
    
    def calculate_agents_erlang_c(self, volume, aht, service_level, asa, interval=3600, max_occupancy=0.85):
        traffic_intensity = (volume * aht) / interval
        num_agents = int(traffic_intensity) + 1
        
        while True:
            prob_wait = self.erlang_c(traffic_intensity, num_agents)
            service_level_calculated = 1 - prob_wait * math.exp(-(num_agents - traffic_intensity) * asa / aht)
            
            if service_level_calculated >= service_level:
                break
            num_agents += 1
        
        occupancy = self.calculate_occupancy(traffic_intensity, num_agents)
        while occupancy > (max_occupancy * 100):
            num_agents += 1
            occupancy = self.calculate_occupancy(traffic_intensity, num_agents)
        
        return num_agents, occupancy

    def calculate_agents_linear(self, volume, aht, efficiency, interval=3600, max_occupancy=0.85):
        workload_hours = (volume * aht) / 3600
        num_agents = math.ceil(workload_hours / (interval / 3600) / efficiency)
        
        traffic_intensity = (volume * aht) / interval
        occupancy = self.calculate_occupancy(traffic_intensity, num_agents)
        while occupancy > (max_occupancy * 100):
            num_agents += 1
            occupancy = self.calculate_occupancy(traffic_intensity, num_agents)
        
        return num_agents, occupancy

    def calculate_required_hc(self, channel, volume, aht, service_level, asa, concurrency, efficiency, shrinkage, interval, max_occupancy):
        if channel.lower() == 'voice':
            base_agents, occupancy = self.calculate_agents_erlang_c(volume, aht, service_level, asa, interval, max_occupancy)
        elif channel.lower() == 'chat':
            adjusted_aht = aht * (1 + 0.1 * (concurrency - 1))
            base_agents, occupancy = self.calculate_agents_erlang_c(volume, adjusted_aht, service_level, asa, interval, max_occupancy)
            base_agents = math.ceil(base_agents / concurrency)
            traffic_intensity = (volume * adjusted_aht) / interval
            occupancy = self.calculate_occupancy(traffic_intensity / concurrency, base_agents)
        elif channel.lower() == 'email':
            base_agents, occupancy = self.calculate_agents_linear(volume, aht, efficiency, interval, max_occupancy)
        else:
            raise ValueError("The channel must be 'voice', 'chat', or 'email'")

        final_hc = math.ceil(base_agents / (1 - shrinkage))
        return final_hc, occupancy

if __name__ == "__main__":
    root = tk.Tk()
    app = HeadcountCalculatorApp(root)
    root.mainloop()